In [ ]:
#Importamos librerias
import requests
import json
from IPython.display import JSON


#Importamos la variable flights_key que esta dentro del archivo keys
from keys import *

In [ ]:
#Guarda la dirección (URL) de la API que da información de los vuelos
url = "https://aerodatabox.p.rapidapi.com/flights/airports/icao/ENGM/2025-09-01T08:00/2025-09-01T20:00"

#Son los parametros que se envian
querystring = {"withLeg":"true","direction":"Arrival","withCancelled":"true","withCodeshared":"true","withCargo":"true","withPrivate":"true","withLocation":"false"}

#Define los encabezados para acceder a la API
headers = {
    'x-rapidapi-host': "aerodatabox.p.rapidapi.com",
    'x-rapidapi-key': flights_key
    }

#Hace la peticion a la API con el metodo GET
response = requests.request("GET", url, headers=headers, params=querystring)

In [8]:
#Verifica si la respuesta de la API fue existosa gracias al codigo 200
if response.status_code == 200:

    #Convierte en formato JSON
    new_data = response.json()

    #Abre el archivo data.json para leer los datos ya guardados 
    try:
        with open("data.json", "r") as json_file:
            existing_data = json.load(json_file)
    #Si el archivo no existe crea una lista vacia
    except (FileNotFoundError, json.decoder.JSONDecodeError):
        existing_data = []

    #Agrega los nuevos datos al final de la lista
    existing_data.append(new_data)

    #vuelve a abrir el archivo data.json para escribir
    with open("data.json", "w") as json_file:
        json.dump(existing_data, json_file, indent=4)
        print("Data appended to data.json file.")

#Si la respuesta no fue exitosa muestra un mensaje de error
else:
    print("Failed to retrieve data from the API. Status code:", response.status_code)

Data appended to data.json file.


In [ ]:
#Convierte la respuesta de la API que viene en JSON originalmente
flight_arrivals = response.json()

for flight in flight_arrivals:
    flight_arrivals['arrivals'][0]['arrival']['scheduledTime'] #Hora de llegada
    flight_arrivals['arrivals'][0]['number'] #Numero de vuelo
    flight_arrivals['arrivals'][0]['arrival'] #Info de llegada
    flight_arrivals['arrivals'][0]['departure']['airport']['name'] #Aeropuerto de salida
    flight_arrivals['arrivals'][0]['aircraft']['model'] #Modelo del avion
    flight_arrivals['arrivals'][0]['airline']['name'] #Nombre de la aerolinea

In [ ]:
#Importamos librerias
import pandas as pd
from numpy import nan

#Gardamos la respuesta de la API en jason
flight_arrivals = response.json()

#Hacemos una funcion que toma ls datos de un vuelo y los convierte en fila
def flight_extraction(flights):
    #Convierte los datos del vuelo en formto de fila
    flights_df = pd.json_normalize({
        'Scheduled_arrival_time': flights['arrival']['scheduledTime'], #Hora de llegada
        'flight_number': flights['number'], #Numero de vuelo
        'from' : flights['departure']['airport']['name'], #Aeropuerto de salida
        'airline' : flights['airline']['name'], #Nombre de la aerolinea
    })
    return(flights_df) #Devolvemos como DataFrame

#Aplicamos la funcion a cada vuelo y juntamos las filas en una sola tabla
flight_arrivals = pd.concat(
    [flight_extraction(flight) for flight in flight_arrivals['arrivals']]
)

#Mostramos las primeras 5 filas de la tabla
flight_arrivals.head()

,flight_number,from,airline,Scheduled_arrival_time.utc,Scheduled_arrival_time.local
0,DY 361,Evenes,Norwegian Air Shuttle,2025-09-01 06:20Z,2025-09-01 08:20+02:00
0,SK 454,Copenhagen,SAS,2025-09-01 06:10Z,2025-09-01 08:10+02:00
0,DY 523,Stavanger,Norwegian Air Shuttle,2025-09-01 06:20Z,2025-09-01 08:20+02:00
0,DY 605,Bergen,Norwegian Air Shuttle,2025-09-01 06:25Z,2025-09-01 08:25+02:00
0,D8 3220,Copenhagen,Norwegian Air Sweden,2025-09-01 06:20Z,2025-09-01 08:20+02:00
